In [208]:
import pandas as pd
from helpers import *
import os
from collections import Counter

In [209]:
NUM_OF_PUB = 284

In [210]:
OUTPUT = 'eval_output'
if not os.path.exists(OUTPUT):
    os.makedirs(OUTPUT)

## Evaluate decisions made by the keyword approach versus the silver standard

In [211]:
decision_keyword = pd.read_csv('output/decision_df_keywords.csv')
annotation = pd.read_excel('data/Annotation_JCDL2024_YF.xlsx', sheet_name='284')

In [212]:
decision_keyword.head()

,DOI,decision,rationale,stage
0,10.1002/chem.201601150,negligible risk,This paper is a review,1
1,10.1002/chem.202004790,negligible risk,This paper is a review,1
2,10.1002/mrc.4989,negligible risk,This paper is a review,1
3,10.1002/mrc.5234,negligible risk,This paper is a review,1
4,10.1002/mrc.5260,negligible risk,This paper is a review,1


In [213]:
decision_keyword = decision_keyword[['DOI','decision', 'stage']]
annotation = annotation[['DOI','Silver Standard']]
decision_keyword['DOI'] = decision_keyword['DOI'].apply(clean)
annotation['DOI'] = annotation['DOI'].apply(clean)

In [214]:
# eval_df = pd.merge(left=decision_keyword, right=annotation, on='DOI', how='left')
# eval_df.to_csv('eval_output/eval_keyword_silver.csv', index=False)

## Evaluate decisions made by the ML approach versus the silver standard

In [215]:
decision_df_tilo = pd.read_csv('output/decision_df_tilo.csv')

In [216]:
decision_df_tilo = decision_df_tilo[['DOI','decision', 'stage']]
decision_df_tilo['DOI'] = decision_df_tilo['DOI'].apply(clean)
decision_df_tilo.head()

,DOI,decision,stage
0,10.1002/anie.201706532,high risk,3
1,10.1002/anie.201708266,negligible risk,3
2,10.1002/anie.201810566,high risk,3
3,10.1002/anie.201902777,negligible risk,3
4,10.1002/anie.201916566,high risk,3


In [217]:
decision_df_tilo.loc[decision_df_tilo['decision'].isna()] = 'negligible risk'

In [218]:
# eval_df = pd.merge(left=decision_df_tilo, right=annotation, on='DOI', how='left')
# eval_df.to_csv('eval_output/eval_tilo_silver.csv', index=False)

## Merge data for all decisions

In [219]:
combined_df = pd.merge(left=annotation, right=decision_keyword, on='DOI', how='left').merge(right=decision_df_tilo.drop(columns=['stage']), on='DOI', how='left', suffixes=('_keyword', '_machine_learning'))

combined_df['decision_base'] = ['high risk']*combined_df.shape[0]

for idx in range(len(combined_df)):
    if combined_df['stage'][idx] == 1 or combined_df['stage'][idx] == 2:
        combined_df['decision_base'][idx] = combined_df['decision_keyword'][idx]
        combined_df['decision_machine_learning'][idx] = combined_df['decision_keyword'][idx]

C:\Users\yuanx\AppData\Local\Temp\ipykernel_5660\2581704936.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['decision_base'][idx] = combined_df['decision_keyword'][idx]
C:\Users\yuanx\AppData\Local\Temp\ipykernel_5660\2581704936.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['decision_machine_learning'][idx] = combined_df['decision_keyword'][idx]
C:\Users\yuanx\AppData\Local\Temp\ipykernel_5660\2581704936.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

### produced results shown in Table 3 of the manuscript

In [220]:
# conversion
combined_df['decision_keyword_conv'] = [conversion(silver, decision) for silver, decision in zip(combined_df['Silver Standard'], combined_df['decision_keyword'])]

combined_df['decision_machine_learning_conv'] = [conversion(silver, decision) for silver, decision in zip(combined_df['Silver Standard'], combined_df['decision_machine_learning'])]

combined_df['decision_base_conv'] = [conversion(silver, decision) for silver, decision in zip(combined_df['Silver Standard'], combined_df['decision_base'])]

In [221]:
combined_df.to_csv('output/combined_df.csv', index=False)

In [222]:
counter = Counter(combined_df['decision_base_conv'])

In [223]:
counter

Counter({'TP': 85, 'FP': 118, 'TN': 78, 'FN': 3})

In [224]:
false_positive_rate = round(counter['FP']/(counter['FP']+counter['TN']), 3)
false_positive_rate

0.602

In [225]:
false_negative_rate = round(counter['FN']/(counter['FN']+counter['TP']), 3)
false_negative_rate

0.034

In [226]:
x = combined_df.loc[combined_df['stage'] == 3, 'decision_keyword_conv']
x = list(x)
counter = Counter(x)
false_positive_rate = round(counter['FP']/(counter['FP']+counter['TN']), 3)
false_positive_rate

0.686

In [227]:
false_negative_rate = round(counter['FN']/(counter['FN']+counter['TP']), 3)
false_negative_rate

0.153

In [228]:
x = combined_df.loc[combined_df['stage'] == 3, 'decision_machine_learning_conv']
x = list(x)
counter = Counter(x)
false_positive_rate = round(counter['FP']/(counter['FP']+counter['TN']), 3)
false_positive_rate

0.178

In [229]:
false_negative_rate = round(counter['FN']/(counter['FN']+counter['TP']), 3)
false_negative_rate

0.129